In [4]:
from stratus_endpoint.handler.base import TaskHandle, TaskResult
from typing import Sequence, List, Dict, Mapping, Optional, Any
import time
from stratus.app.core import StratusCore
import os
import xarray as xa
import pandas as pd
import pdb
import numpy as np
from datetime import date, datetime
from dateutil.rrule import rrule, DAILY, MONTHLY

if __name__ == "__main__":
    start = time.time()

    #  Startup a Stratus zmq client and connect to a server on localhost

    settings = dict(stratus=dict(type="zeromq", client_address="127.0.0.1", request_port="4566", response_port="4567"))
    stratus = StratusCore(settings)
    client = stratus.getClient()

    #M03_dir = "/Users/lakshmipriyanka/Project/MODIS_Aggregation/resources/data/input_data_sample/MYD03/"
    #M06_dir = "/Users/lakshmipriyanka/Project/MODIS_Aggregation/resources/data/input_data_sample/MYD06/"

    # requestSpec = dict(
    #     input=dict(path1=M03_dir, path2=M06_dir, name1=["Latitude", "Longitude"], name2=["Cloud_Mask_1km"]),
    #     operation=[dict(name="xop:cloudFraction")]
    # )

    #arg_string = "data_path.csv 2008/01/01 2008/01/01 [-90,90,-180,180] [1,1] [5] 1 1 1 1 1 1 1 input_file.csv input_Jhist.csv"

    print("\n\n in zmq client")

    df_datapath = pd.read_csv('data_path.csv',  header=0, delim_whitespace=True)
    df_output_path = pd.read_csv('data_path.csv',  header=3, delim_whitespace=True)
    df_varlist = pd.read_csv('input_file.csv',  header=0, delim_whitespace=True)
    df_jvarlist = pd.read_csv('input_Jhist.csv',  header=0, delim_whitespace=True)
    str_start_date='2008/01/01'
    str_end_date='2008/01/01'               


    requestSpec = dict(
        input=dict(data_path_file=df_datapath.to_json(), 
                   output_path = df_output_path.to_json(),
                   start_date=str_start_date, 
                   end_date=str_end_date, 
                   poly='[-90,90,-180,180]', 
                   grid='[1,1]', 
                   spl_num='[5]',
                   sts_switch='1, 1, 1, 1, 1, 1, 1',
                   varlist=df_varlist.to_json(),
                   jvarlist=df_jvarlist.to_json()
                   ),
        operation=[dict(name="xop:modisAggr")]
    )
 

    # Submit the request to the server and wait for the result
    task: TaskHandle = client.request(requestSpec, input)
    result: Optional[TaskResult] = task.getResult(block=True)

    # Display the result

    print("\n\nCompleted computation in " + str(time.time() - start) + " seconds")

    result_file = result.data[0] 


    #ouput dir and file name
    output_dir = np.array(df_output_path)[0,0]
    output_prefix = np.array(df_output_path)[0,1]
    
    if os.path.exists(output_dir) is False:
        os.makedirs(output_dir)

    start_date = np.fromstring(str_start_date, dtype=np.int, sep='/' )
    end_date   = np.fromstring(str_end_date, dtype=np.int, sep='/' )
    start = date(start_date[0], start_date[1], start_date[2])
    until = date(end_date[0], end_date[1], end_date[2])

    for dt in rrule(DAILY, interval=1, dtstart=start, until=until):
        year  = np.int(dt.strftime("%Y"))
        month = np.int(dt.strftime("%m"))

    l3name  = output_prefix + '.A{:04d}{:02d}'.format(year,month)
    subname = '_baseline_monthly_v8_client.h5'
    # subname2 = '_baseline_monthly_v8_client.nc'

    # result_file.to_netcdf(os.path.join(output_dir,'baseline_monthly_v8_client.h5'), mode='w', format='NETCDF4')
    # import pdb; pdb.set_trace()
    result_file.to_netcdf(os.path.join(output_dir,l3name+subname), mode='w', format='NETCDF4')
    ## use hdf5 format and not netcdf4 -- Check a different API with different format
    


    print("Result saved at " + output_dir)


2021-02-25 12:29:49,166 - stratus - INFO - 5434 - 140647525619520 - Adding constructors for packages ['stratus.app', 'stratus.endpoints', 'stratus.handlers', 'stratus.handlers.broker', 'stratus.handlers.celery', 'stratus.handlers.celery.tests', 'stratus.handlers.celery.tests.rest', 'stratus.handlers.celery.tests.zmq', 'stratus.handlers.endpoint', 'stratus.handlers.endpoint.samples', 'stratus.handlers.endpoint.samples.EOF', 'stratus.handlers.endpoint.samples.MODIS', 'stratus.handlers.endpoint.samples.MODIS_Series', 'stratus.handlers.endpoint.samples.MODIS_aggregation', 'stratus.handlers.endpoint.samples.MODIS_data_download', 'stratus.handlers.endpoint.samples.MODIS_with_download', 'stratus.handlers.endpoint.samples.xarrayOps', 'stratus.handlers.endpoint.tests', 'stratus.handlers.lambda', 'stratus.handlers.openapi', 'stratus.handlers.openapi.wps', 'stratus.handlers.rest', 'stratus.handlers.rest.api', 'stratus.handlers.rest.api.core', 'stratus.handlers.rest.api.ows_wps', 'stratus.handlers

Initialized zmq client at:
['  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)\n\n', '  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)\n\n', '  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()\n\n', '  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance\n    app.start()\n\n', '  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 612, in start\n    self.io_loop.start()\n\n', '  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 149, in start\n    self.asyncio_loop.run_forever()\n\n', '  File "/home/supriya/anaconda3/envs/stratus/lib/python3.6/asyncio/base_ev

2021-02-25 12:29:50,961 - stratus - INFO - 5434 - 140647525619520 - EndpointSpecs: {'epas': ['xop*'], 'rid': 'CMLyWZ'}
2021-02-25 12:29:51,132 - stratus - INFO - 5434 - 140647525619520 - [ZP] Sending authenticated exe request {"input": {"data_path_file": "{\"Data_input_path\":{\"0\":\"\\/umbc\\/xfs1\\/cybertrn\\/common\\/Data\\/Satellite_Observations\\/MODIS\\/version_61\\/MYD06_L2\\/\",\"1\":\"\\/umbc\\/xfs1\\/cybertrn\\/common\\/Data\\/Satellite_Observations\\/MODIS\\/version_61\\/MYD03\\/\",\"2\":\"Data_output_path\",\"3\":\"\\/home\\/supriya\\/IS_Thesis\\/stratus_endpoint\\/stratus\\/stratus\\/handlers\\/endpoint\\/samples\\/MODIS_aggregation\\/data\\/zmq_test\\/result\\/\"},\"File_prefix_name\":{\"0\":\"MYD06_L2.A\",\"1\":\"MYD03.A\",\"2\":\"File_prefix_name\",\"3\":\"MYD08_M3\"}}", "output_path": "{\"Data_output_path\":{\"0\":\"\\/home\\/supriya\\/IS_Thesis\\/stratus_endpoint\\/stratus\\/stratus\\/handlers\\/endpoint\\/samples\\/MODIS_aggregation\\/data\\/zmq_test\\/result\\/\"},



 in zmq client


2021-02-25 12:29:52,717 - stratus - INFO - 5434 - 140646655248128 - [RM] Connected response socket on port 4567 with subscription (client/request) id: 'KkOVfp', active = True
2021-02-25 12:29:52,723 - stratus - INFO - 5434 - 140646655248128 - [RM] Awaiting responses
2021-02-25 12:50:02,722 - stratus - INFO - 5434 - 140646655248128 - Caching result: {'type': 'xarray', 'status': 'Status.COMPLETED'}
2021-02-25 12:50:02,723 - stratus - INFO - 5434 - 140646655248128 - [RM] [KkOVfp]: Received response: {'type': 'xarray', 'status': 'Status.COMPLETED'}, new status = Status.COMPLETED, exception = None
2021-02-25 12:50:02,729 - stratus - INFO - 5434 - 140646655248128 - [RM] Awaiting responses




Completed computation in 1213.5657601356506 seconds
Result saved at /home/supriya/IS_Thesis/stratus_endpoint/stratus/stratus/handlers/endpoint/samples/MODIS_aggregation/data/zmq_test/result/
